# Band
The notebook created after the *band* module.

Demonstrates domain classes, methods and functions related to the concept of musical band, as well as the mandatory `__init__()`, `__str__()` and `__eq__()` methods. The `__eq__()` method notes the problem of comparing unhashable objects.

Introduces static fields/attributes and static methods (`@staticmethod`).

Illustrates iterators and generators.

Shows how to serialize `Band` objects to JSON.

## Setup / Data

In [5]:
from datetime import date, datetime, time
from pathlib import Path
import json
import sys
import pickle

from music.musician import Musician
from util.utility import format_date, get_data_dir, get_project_dir
from settings import PROJECT_DIR, DATA_DIR

from testdata.musicians import *

Paul McCartney (solo musician)
Paul McC (band member)
True
1942
Paul McC playing I Saw Her Standing There - One, two, three, four! - ... (playing) ... Thank You! We Love You!
Paul McC playing I Saw Her Standing There... (playing) ... Thank You! We Love You!
Paul McC playing I Saw Her Standing There... (playing) ... Thank You! We Love You!
2
1
<class 'int'>
int
['__new__', '__repr__', '__hash__', '__getattribute__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__add__', '__radd__', '__sub__', '__rsub__', '__mul__', '__rmul__', '__mod__', '__rmod__', '__divmod__', '__rdivmod__', '__pow__', '__rpow__', '__neg__', '__pos__', '__abs__', '__bool__', '__invert__', '__lshift__', '__rlshift__', '__rshift__', '__rrshift__', '__and__', '__rand__', '__xor__', '__rxor__', '__or__', '__ror__', '__int__', '__float__', '__floordiv__', '__rfloordiv__', '__truediv__', '__rtruediv__', '__index__', 'conjugate', 'bit_length', 'bit_count', 'to_bytes', 'from_bytes', 'as_integer_ratio', '__tru

NameError: name 'john' is not defined

## The *Band* class
The class describing the concept of musical band. It is assumed that a musician is sufficiently described by their name and whether they are a solo musician or a member of a band. It includes a list of `Musician` objects (band members) and the dates when the band started/stopped performing together.

Introduces `@staticmethod`s, iterators (the `__iter__()` and `__next__()` methods) and generators. Implicitly introduces exception handling as well (in demonstrating iterators and generators).

In [6]:
class Band():
    """The class describing the concept of a music group/band.
    It includes a list of Musician objects (band members)
    and the dates when the band started/stopped performing together.
    """

    # Class variables: like static fields in Java; typically defined and initialized before __init__()
    # Insert a class variable (static field), such as genres, date_pattern,...

    genres = ['rock', 'blues', 'pop', 'alternative', 'unknown']

    def __init__(self, name, *members, start=date.today(), end=date.today()):
        # pass                                            # introduce and initialize iterator counter, self.__i
        self.name = name
        self.members = members
        self.start = start
        self.end = end
        self.__i = 0

    def __str__(self):
        n = self.name
        m = f'({", ".join([member.name for member in self.members]) if self.members else "members unknown"})'
        s = self.start.year
        e = self.end.year
        return f'{n} {m}, {s}-{e}'

    def __eq__(self, other):
        # Musician objects are unhashable, so comparing the members tuples from self and other directly does not work;
        # see https://stackoverflow.com/a/14721133/1899061, https://stackoverflow.com/a/17236824/1899061
        # return self == other if isinstance(other, Band) else False    # No! Musician objects are unhashable!
        # However, this works:
        # return self.__dict__ == other.__dict__ if isinstance(other, Band) else False

        t = isinstance(other, Band)
        n = self.name == other.name

        # # members must be compared 'both ways', because the two tuples can be of different length
        # m_diff_1 = [x for x in self.members if x not in other.members]
        # m_diff_2 = [x for x in other.members if x not in self.members]
        # m = len(m_diff_1) == len(m_diff_2) == 0

        # members must be compared 'both ways', because the two tuples can be of different length
        m = all([x in self.members for x in other.members]) and all([x in other.members for x in self.members])

        y = (self.start == other.start) and (self.end == other.end)
        return t and n and m and y

    @staticmethod
    def is_date_valid(d):
        """It is assumed that a band does not perform together since more than ~60 years ago.
        So, the valid date to denote the start of a band's career is between Jan 01, 1960, and today.
        """

        return date(1954, 7, 5) <= d <= date.today() if isinstance(d, date) else False

    def __iter__(self):
        """Once __iter__() and __next__() are implemented in a class,
        we can create an iterator object by calling the iter() built-in function on an object of the class,
        and then call the next() built-in function on that object.
        It is often sufficient to just return self in __iter__(),
        if the iterator counter such as self.__i is introduced and initialized in __init__().
        Alternatively, the iterator counter (self.__i) is introduced and initialized here.
        """

        self.__i = 0
        return self               # sufficient if the iterator counter is introduced and initialized in __init__()

    def __next__(self):
        if self.__i < len(self.members):
            next_m = self.members[self.__i]
            self.__i += 1
            return next_m
        else:
            raise StopIteration

#### The basic methods
Check the basic methods (`__init__()`, `__str__()`,...) of the `Band` class.

In [7]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
print(theBeatles)
beatles = Band('The Beatles', *[johnLennon, georgeHarrison, paulMcCartney, ringoStarr, Musician('Pete Best')],
               start=date(1957, 7, 6), end=date(1970, 4, 10))
print(theBeatles == beatles)

NameError: name 'johnLennon' is not defined

#### Class variables
Python class variables are much like `static` fields in Java. They are typically defined and initialized before `__init__()`.

In [ ]:
print(Band.genres)

#### Class methods
Check the date validator (the `@staticmethod` `is_date_valid(<date>)`).

In [ ]:
print(Band.is_date_valid(date(1957, 7, 6)))

#### Iterators
Check the iterator defined in the `Band` class. After the iterator is exhausted, repeated attempts to run the iterator fail before the iterator is re-initialized.

In [ ]:
for _ in range(len(theBeatles.members)):
    print(theBeatles.__next__().name)
# print(theBeatles.__next__().name)             # iterator exhausted, must be re-initiated
print()

# i = iter(theBeatles)
# print(i)
i = iter(theBeatles)                            # re-initiating the iterator
for _ in range(len(theBeatles.members)):
    print(next(theBeatles))
print()

# # Alternatively
# theBeatles.__iter__()
# for _ in range(len(theBeatles.members)):
#     print(theBeatles.__next__().name)
# print()

# Repeated attempt to run the iterator fails, because the iterator is exhausted

#### Generators
Develop a simple generator to demonstrate generator objects and the `yield` command.

In [ ]:
def next_member(band):
    """Generator that shows members of a band, one at a time.
    yield produces a generator object, on which we call the next() built-in function.
    A great tutorial on generators: https://realpython.com/introduction-to-python-generators/.
    """

    for member in band.members:
        input('Next: ')
        yield member
        print('Yeah!')

In [ ]:
member_generator = next_member(theBeatles)
print(member_generator)
print(type(member_generator))
print()

while True:
    try:
        print(next(member_generator))
    except:
        break

#### Generator expressions

In [ ]:
print(i**2 for i in range(4))
print(next((i**2 for i in range(4))))
print(next((i**2 for i in range(4))))
ge = (i**2 for i in range(4))
print(ge)
print(next(ge))     # 0
print(next(ge))     # 1
print(next(ge))     # 4
print(next(ge))     # 9
print(next(ge))     # raises StopIteration

## JSON
The traditional way to work with JSON would be something like the following class and functions:

In [ ]:
class BandEncoder(json.JSONEncoder):
    """JSON encoder for Band objects (cls= parameter in json.dumps()).
    """

    def default(self, band):
        # recommendation: always use double quotes with JSON

        pass


def band_py_to_json(band):
    """JSON encoder for Band objects (default= parameter in json.dumps()).
    """


def band_json_to_py(band_json):
    """JSON decoder for Band objects (object_hook= parameter in json.loads()).
    """

Demonstrate JSON encoding/decoding of simple data types.
Refer to [this section in the Standard Library](https://docs.python.org/3.3/library/json.html#encoders-and-decoders) for details.

In [ ]:
d = json.dumps({'one': [1, True, 'Uno'], 'two': (2, 3, 4)}, indent=4)
print(d)
l = json.loads(d)
print(l)

It is much more complicated with classes, objects, dates, etc. However, with the `json_tricks` module (from the external `json-tricks` library) the things are largely simplified.

From the [json-tricks external package](https://github.com/mverleg/pyjson_tricks) documentation:

The JSON string resulting from applying the `json_tricks.dumps()` function stores the module and class name. The class must be importable from the same module when decoding (and should not have changed). If it isn't, you have to manually provide a dictionary to `cls_lookup_map` when loading in which the class name can be looked up. Note that if the class is imported, then `globals()` is such a dictionary (so try `loads(json, cls_lookup_map=glboals()))`.

Also note that if the class is defined in the 'top' script (that you're calling directly), then this isn't a module and the import part cannot be extracted. Only the class name will be stored; it can then only be deserialized in the same script, or if you provide `cls_lookup_map`.

That's why the following warning appears when serializing Band objects in this script:

UserWarning: class <class '__main__.Musician'> seems to have been defined in the main file;
unfortunately this means that it's module/import path is unknown,
so you might have to provide cls_lookup_map when decoding.


JSON encoding of `Musician` objects using `json_tricks`:

In [ ]:
from json_tricks import loads, dumps

In [ ]:
# Single object
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
theBeatles_json = dumps(theBeatles, indent=4)
print(theBeatles_json)
print(theBeatles == loads(theBeatles_json))

In [ ]:
# List of objects
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
theRollingStones = Band('The Rolling Stones', *[mickJagger, keithRichards, ronWood, charlieWatts],
                        start=date(1962, 7, 12))
pinkFloyd = Band('Pink Floyd', *[sydBarrett, davidGilmour, rogerWaters, nickMason, rickWright])

bands_json = dumps([theBeatles, theRollingStones, pinkFloyd], indent=4)
print(bands_json)
print([theBeatles, theRollingStones, pinkFloyd] == loads(bands_json))

# Exception handling
Base class:

In [ ]:
class BandError(Exception):
    """Base class for exceptions in this module.
    """

    pass

A simple user-defined exception:

In [ ]:
class BandNameError(BandError):
    """Exception raised when the name of a band is specified incorrectly.
    """

    def __init__(self, name):
        self.name = name
        self.message = f'BandNameError: \'{self.name}\' is not a valid band name'

#### Catching exceptions - *try-except* block

In [ ]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
try:
    for i in range(5):
        print(theBeatles.members[i])
except Exception as err:
    print()
    # print(err)
    # sys.stderr.write('\n' + str(type(err)) + ': ' + err.args[0] + '\n')
    sys.stderr.write(f'\n{type(err).__name__}: {err.args[0]}\n\n')

#### Catching multiple exceptions and the *finally* clause

In [ ]:
# Catching multiple exceptions and the 'finally' clause
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
try:
    for i in range(4):
        print(theBeatles.members[i])
    print(theBeatles / 4)
except IndexError as err:
    print()
    sys.stderr.write(f'\n{type(err).__name__}: {err.args[0]}\n\n')
except Exception as err:
    print()
    sys.stderr.write(f'\nCaught an exception: {type(err).__name__}: {err.args[0]}\n\n')
finally:
    print('Caught an exception. Stopped any further processing.')

#### Using the *else* clause
If present in the `try-except` block, the `else` clause must be after all `except` clauses.

In [ ]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
try:
    for i in range(4):
        print(theBeatles.members[i])
except IndexError as err:
    print()
    sys.stderr.write(f'\n{type(err).__name__}: {err.args[0]}\n\n')
else:
    print(f'\nThat\'s all.')

#### Catching 'any' exception
An empty `except` clause catches any exception that can possibly be raised in the `try` block.

In [ ]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
try:
    for i in range(5):
        print(theBeatles.members[i])
except:
    print()
    sys.stderr.write(f'\nCaught an exception\n\n')

#### Catching user-defined exceptions

In [ ]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
try:
    band = Band('')
except Exception as err:
    print()
    #     sys.stderr.write(f'\n{type(err).__name__}: {err.args[0]}\n\n')
    # sys.stderr.write(f'\n{type(err).__name__}:\n{err.message}\n\n')
    sys.stderr.write(f'\n{err.message}\n\n')

# Working with files
For some reasons, `get_project_dir()` and `get_data_dir()` from `util.utility` return `None` when called from here. Thus `DATA_DIR` is imported from *settings.py* and used here.

#### Writing to a text file
`<outfile>.write(str(<obj>)`, `<outfile>.writelines([str(<obj>)+'\n' for <obj> in <objs>]`

In [ ]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr],
                      start=date(1957, 7, 6), end=date(1970, 4, 10))
theRollingStones = Band('The Rolling Stones', *[mickJagger, keithRichards, ronWood, charlieWatts],
                        start=date(1962, 7, 12))
pinkFloyd = Band('Pink Floyd', *[sydBarrett, davidGilmour, rogerWaters, nickMason, rickWright])
bands = [theBeatles, theRollingStones, pinkFloyd]

In [ ]:
# print(type(get_data_dir()))                 # get_data_dir() returns None for some reasons and is not used here
file = DATA_DIR / 'bands.txt'
with open(file, 'w') as f:
    # for b in bands:
    #     f.write(str(b) + '\n')
    f.writelines([str(b) + '\n' for b in bands])
print('Done')

#### Reading from a text file
`<infile>.read()`, `<infile>.readline()`

In [ ]:
# print(type(get_data_dir()))                 # get_data_dir() returns None for some reasons and is not used here
file = DATA_DIR / 'bands.txt'
with open(file, 'r') as f:
    # lines = f.read().rstrip()             # rstrip() removes an extra '\n' in the end
    lines = ''
    while True:
        line = f.readline()
        if line:
            lines += line
        else:
            break
print(lines.rstrip())                       # rstrip() removes an extra '\n' in the end
print('Done')

#### Writing to a binary file
`pickle.dump(<obj>, <outfile>)`


In [ ]:
# print(type(get_data_dir()))                 # get_data_dir() returns None for some reasons and is not used here
file = DATA_DIR / 'bands.txt'
bands = [theBeatles, theRollingStones, pinkFloyd]
with open(file, 'wb') as f:
    pickle.dump(bands, f)
print('Done')

#### Reading from a binary file
`pickle.load(<infile>)`

In [ ]:
# print(type(get_data_dir()))                 # get_data_dir() returns None for some reasons and is not used here
file = DATA_DIR / 'bands.txt'
with open(file, 'rb') as f:
    bands_from_file = pickle.load(f)
print('Done')
for band in bands_from_file:
    print(band)